In [1]:
# trade_nse.py
from z_helper import *
util.startLoop()

# from json
a = assign_var('nse') + assign_var('common')
for v in a:
    exec(v)
    
ib = get_connected('nse', 'live')

In [2]:
# from portfolio
#_______________

p = util.df(ib.portfolio()) # portfolio table

# extract option contract info from portfolio table
dfp = pd.concat([p, util.df([c for c in p.contract])[util.df([c for c in p.contract]).columns[:7]]], axis=1).iloc[:, 1:]
dfp = dfp.rename(columns={'lastTradeDateOrContractMonth': 'expiration'})

# get the total position
dfp1 = dfp.groupby('symbol').sum()['position']

# from options pickle
#____________________

# get the options
df_opt = pd.read_pickle(fspath+'sized.pkl')
df_opt = df_opt.assign(und_remq=(nse_assignment_limit/(df_opt.lot*df_opt.undPrice)).astype('int')) # remaining quantities in entire nse

# compute the remaining quantities
df_opt1 = df_opt.groupby('symbol').first()[['lot', 'margin', 'und_remq']]

df_opt2 = df_opt1.join(dfp1).fillna(0).astype('int')
df_opt2 = df_opt2.assign(remqty=df_opt2.und_remq+(df_opt2.position/df_opt2.lot).astype('int'))

dfrq = df_opt2[['remqty']]

In [3]:
# remove nan in margins and close prices. These could be dead ones.
df_opt = df_opt[~df_opt.margin.isnull()].reset_index(drop=True)
df_opt = df_opt[~df_opt.close.isnull()].reset_index(drop=True)

# remove margins with 1e7
df_opt = df_opt[df_opt.margin < 1e7]

# establish quantity and minimum expected price and sdmult
df_opt = df_opt.assign(qty=1, expPrice = df_opt.close+0.1, sd=abs(df_opt.strike-df_opt.undPrice)/df_opt.stDev)

# recacluate rom based on expPrice upgrade
df_opt = df_opt.assign(rom=df_opt.expPrice/df_opt.margin*365/df_opt.dte*df_opt.lot)

# calculate the standard deviation change (lowest is most risky)
df_opt = df_opt.sort_values('sd', ascending=True)

# for those not meeting minimum expected ROM, up the expected price
rom_mask = (df_opt.rom < minexpRom)
df_opt.loc[rom_mask, 'expPrice'] = ((df_opt[rom_mask].expPrice * minexpRom )/ df_opt[rom_mask].rom).apply(lambda x: get_prec(x, prec))

In [4]:
# pd.options.display.float_format = '{:,.2f}'.format
cols = ['symbol', 'dte', 'lot', 'right', 'undPrice', 'strike', 'stDev', 'lo52', 'hi52', 'margin', 'close', 'rom','sd','expPrice']
df_opt[cols]

,symbol,dte,lot,right,undPrice,strike,stDev,lo52,hi52,margin,close,rom,sd,expPrice
19,ADANIPOWE,18,20000,P,65.70,48.0,6.859537,15.450,65.750,527753.57,0.45,0.422651,2.580349,1.30
407,TITAN,53,750,P,1297.25,1100.0,74.700226,749.500,1334.700,174710.75,2.85,0.087213,2.640554,33.85
253,LT,53,375,P,1563.20,1320.0,92.004282,1197.450,1596.000,106573.62,2.20,0.055735,2.643355,41.25
20,ADANIPOWE,18,20000,P,65.70,47.5,6.859537,15.450,65.750,527753.57,0.40,0.384228,2.653240,1.30
21,ADANIPOWE,18,20000,P,65.70,47.0,6.859537,15.450,65.750,527753.57,0.35,0.345805,2.726132,1.30
254,LT,53,375,P,1563.20,1300.0,92.004282,1197.450,1596.000,106573.62,1.45,0.037560,2.860736,41.25
22,ADANIPOWE,18,20000,P,65.70,46.0,6.859537,15.450,65.750,527753.57,0.30,0.307383,2.871914,1.30
408,TITAN,53,750,P,1297.25,1080.0,74.700226,749.500,1334.700,174710.75,1.85,0.057649,2.908291,33.85
416,TORNTPOWE,18,3000,P,304.95,235.0,24.049851,214.000,308.550,128728.91,0.65,0.354427,2.908542,2.10
131,CONCOR,53,1563,P,576.70,480.0,32.108510,440.125,579.600,164526.79,0.50,0.039255,3.011663,15.30


In [5]:
tb = trade_blocks(ib, df_opt, exchange)

Peer closed connection
clientId 1 already in use?
API connection failed: CancelledError()


Started to throttle requests
Stopped to throttle requests
Started to throttle requests
Stopped to throttle requests
Started to throttle requests
Stopped to throttle requests
Started to throttle requests
Stopped to throttle requests
Started to throttle requests
Stopped to throttle requests
Started to throttle requests
Stopped to throttle requests
Started to throttle requests
Stopped to throttle requests
Started to throttle requests
Stopped to throttle requests
Started to throttle requests
Stopped to throttle requests
Started to throttle requests
Stopped to throttle requests
Started to throttle requests
Stopped to throttle requests
Started to throttle requests
Stopped to throttle requests
Started to throttle requests
Stopped to throttle requests
Started to throttle requests
Stopped to throttle requests
Started to throttle requests
Stopped to throttle requests
Started to throttle requests
Stopped to throttle requests
Started to throttle requests
Stopped to throttle requests
Started to thr